# Descrizione lessico ELIta
Il lessico comprende 6905 elementi, tra parole ed emoji. 

Il set di dati contiene il: 
* 21% di aggettivi, 
* il 50% di nomi, 
* il 21% di verbi,
* l'8% di parole che possono essere considerate sia aggettivi che nomi. 

Inoltre, è stato incluso un numero minore di avverbi, espressioni (ad esempio, "restare a bocca aperta" significa "guardare a bocca aperta") e interiezioni (ad esempio, "beh", "boh"). 

La raccolta dei dati ha comportato un processo di annotazione che includeva sia l'associazione delle parole alle emozioni 8 di base (Plutchik), utilizzando una scala da: 
* "non associato" (0), 
* "debolmente associato" (0,25), 
* "moderatamente associato" (0,75),
* "fortemente associato" (1), 

sia la valutazione degli elementi in base alle 3 dimensioni VAD (valenza, eccitazione e dominanza) utilizzando una scala da 1 a 9. 

Ognuna delle 6905 parole/emoji presenti nel lessico ha da un minimo di 5 a un massimo di 10 annotazioni (in media 5,13 annotazioni per parola).

Il lessico è fornito in diverse versioni:
* **RAW**, contiene **tutte** le annotazioni originali fatte dai partecipanti, senza filtri o aggregazioni.
* **GOLDEN**, versione **pulita** della RAW. Sono state selezionate:
    * le 5 annotazioni più simili tra loro per parola,
    * più 1 riga generata automaticamente, "Golden Standard", calcolata dalla moda delle 5 annotazioni migliori.
* **INTENSITY**, versione **aggregata continua**.\
Ogni parola ha una record per emozione (sono state aggiunte: **amore**, media tra gioia e fiducia, e **neutrale**) e dimensione VAD (13 in totale), con la media delle annotazioni selezionate nella versione GOLDEN.
* **BINARY**, versione **aggregata discreta**.\
Converte le annotazioni continue della versione INTENSITY in valori binari (0 o 1) utilizzando una soglia di 0,5.

# Scelta della versione del lessico per la visualizzazione
Per l'obbiettivo di visualizzazione del lessico ELIta con metodi distribuzionali, è stata scelta la versione **INTENSITY** del lessico. 

La ragione, per questa decisione, è che perché INTENSITY fornisce punteggi continui aggregati, necessari per calcolare le distanze semantiche precise tra le parole nello spazio vettoriale PCA/SVD.

Il file è fornito in modo che ci sia una riga per ogni coppia parola-emozione, contenente la media delle annotazioni.\
  Per l'analisi PCA, questo formato verrà trasformato in una matrice vettoriale dove ogni parola ha un unico vettore di punteggi per le 8 emozioni di base (Gioia, Tristezza, Rabbia, Disgusto, Paura, Fiducia, Sorpresa, Anticipazione) più le dimensioni VAD (Valenza, Attivazione, Dominanza) e le categorie aggiuntive (Amore, Neutrale).

# Caricamento e preparazione dati per l'utilizzo
Importiamo il file ELIta_INTENSITY.txt come un DataFrame di pandas. \
Dato che il file non presenta le intestazioni delle colonne, le definiamo qui manualmente. Ciò faciliterà i passaggi successivi.

In [1]:
import pandas as pd

intensityData = pd.read_csv("ELIta_INTENSITY.txt", sep=',', header=None, names=['parola', 'emozione', 'valore'])
intensityData.head()

,parola,emozione,valore
0,‼,gioia,0.71
1,‼,tristezza,0.12
2,‼,rabbia,0.67
3,‼,disgusto,0.04
4,‼,paura,0.17


## Verifica integrità dati
Controllo valori unici nelle colonne 'parola' ed 'emozione' per confermare che il dataset sia stato caricato correttamente.

In [2]:
unique_words = intensityData['parola'].nunique()
unique_emotions = intensityData['emozione'].nunique()
print(f"Numero di parole uniche: {unique_words}")
print(f"Numero di emozioni/dimensioni uniche: {unique_emotions}")

Numero di parole uniche: 6905
Numero di emozioni/dimensioni uniche: 13


Controllo per valori mancanti nel DataFrame.

In [3]:
total_nulls = intensityData.isnull().sum().sum()
print(f"Totale valori mancanti nel dataset: {total_nulls}")

Totale valori mancanti nel dataset: 0


## Trasformazione del DataFrame in una matrice vettoriale
Manipolazione del dataset in una matrice per avere una riga per ogni parola e una colonna per ogni emozione/dimensione.

In [4]:
pivotedData = intensityData.pivot(index='parola', columns='emozione', values='valore')
pivotedData.head()

emozione,amore,aspettativa,attivazione,disgusto,dominanza,fiducia,gioia,neutrale,paura,rabbia,sorpresa,tristezza,valenza
parola,,,,,,,,,,,,,
????,0.08,0.17,7.00,0.17,4.33,0.04,0.12,0.0,0.17,0.42,0.79,0.42,5.17
a_caso,0.04,0.21,5.00,0.08,3.00,0.04,0.04,0.0,0.54,0.29,0.83,0.21,4.33
a_malincuore,0.00,0.12,5.50,0.21,3.67,0.00,0.00,0.0,0.21,0.21,0.04,0.83,3.33
a_scanso_di,0.21,0.58,5.33,0.04,4.67,0.25,0.17,0.0,0.17,0.42,0.21,0.25,5.00
abbagliante,0.32,0.62,7.00,0.00,6.50,0.17,0.46,0.0,0.12,0.12,0.71,0.04,6.67


## Salvataggio della matrice vettoriale
Salviamo la matrice vettoriale in un file CSV per utilizzi futuri.

In [5]:
pivotedData.to_csv("ELIta_INTENSITY_Matrix.csv")

---

# Visualizzazione tramite Riduzione Dimensionale (PCA e TruncatedSVD).
La matrice a 13 dimensioni (8 emozioni + 2 extra + 3 VAD) appena creata verrà schiacciata in 2 dimensioni per poterla disegnare su un piano 2D.

## Preparazione specifica per la visualizzazione
Prima di dare i dati in pasto agli algoritmi, dobbiamo fare due cose importanti:

1. **Selezionare le feature**: per la visualizzazione delle "parole in base all'emozione", è meglio usare:
    - prima, solo le 8 emozioni di base di Plutchik.
    - sucessivamente, aggiungere le 2 emozioni extra (amore e neutrale).
    - infine, le 3 dimensioni VAD.

    Le analisi verranno fatte separatamente per ogni set di feature. Non calcoliamo la riduzione dimensionale su tutte le feature insieme perché emozioni e dimensioni VAD sono su due scale diverse e mescolarle potrebbe distorcere i risultati.
2. **Gestire le emoji**: il lessico contiene sia parole testuali che emoji. Per una visualizzazione più chiara, è meglio separare le due categorie e fare analisi distinte.
    - Prima analizzeremo solo le parole.
    - Poi faremo un'analisi separata per le emoji.

3. *Calcolare l'emozione dominante**: ci serve per colorare i punti nel grafico (es. colorare di rosso le parole di "rabbia").

In [6]:
basic_emotions = ['gioia', 'fiducia', 'paura', 'sorpresa', 'tristezza', 'disgusto', 'rabbia', 'aspettativa']

X = pivotedData[basic_emotions].fillna(0)

dominant_emotion = X.idxmax(axis=1) # Trova l'emozione con il valore più alto per ogni parola

print("Matrice X creata. Dimensioni:", X.shape)
print("Emozioni dominanti calcolate.")

Matrice X creata. Dimensioni: (6905, 8)
Emozioni dominanti calcolate.


## Separazione dei Dati (Emoji vs Parole)

In [7]:
import emoji

def is_likely_emoji(text):
    return emoji.emoji_count(str(text)) > 0

pivotedData['is_emoji'] = [is_likely_emoji(x) for x in pivotedData.index]

df_only_emojis = pivotedData[pivotedData['is_emoji'] == True].copy()
df_only_words = pivotedData[pivotedData['is_emoji'] == False].copy()

print(f"Totale elementi: {len(pivotedData)}")
print(f"Numero di Emoji trovate: {len(df_only_emojis)}")
print(f"Numero di Parole testuali: {len(df_only_words)}")

Totale elementi: 6905
Numero di Emoji trovate: 186
Numero di Parole testuali: 6719


## Visualizzazione delle Emoji (PCA)


In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

# Utilizzare un font che supporti meglio le emoji
import matplotlib.font_manager as fm

# Trovare un font con supporto emoji
font_path = "/System/Library/Fonts/Apple Color Emoji.ttf"  # Esempio per macOS
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    font_name = fm.FontProperties(fname=font_path).get_name()
    plt.rcParams['font.family'] = font_name

# 1. Preparazione Dati Emoji
basic_emotions = ['gioia', 'fiducia', 'paura', 'sorpresa', 'tristezza', 'disgusto', 'rabbia', 'aspettativa']
X_em = df_only_emojis[basic_emotions].fillna(0)

# Standardizzazione
scaler_em = StandardScaler()
X_em_scaled = scaler_em.fit_transform(X_em)

# 2. PCA sulle Emoji
pca_em = PCA(n_components=2)
coords_em = pca_em.fit_transform(X_em_scaled)

df_pca_em = pd.DataFrame(data=coords_em, columns=['PC1', 'PC2'])
df_pca_em['emoji'] = df_only_emojis.index
# Ricalcoliamo l'emozione dominante solo per questo subset
df_pca_em['emozione_dominante'] = X_em.idxmax(axis=1).values

# 3. Grafico SOLO EMOJI
plt.figure(figsize=(12, 10))

# IMPOSTAZIONE FONT PER MAC (Cruciale per vedere le faccine)
plt.rcParams['font.family'] = 'Arial Unicode MS' 

# Scatterplot base (i punti colorati)
sns.scatterplot(
    x='PC1', y='PC2', 
    hue='emozione_dominante', 
    data=df_pca_em, 
    palette='tab10', 
    s=100, # Punti più grandi
    alpha=0.4 # Trasparenti
)

# Aggiungiamo le Emoji come testo sopra i punti
# Stampiamo TUTTE le emoji perché sono poche (rispetto alle 6000 parole)
for i in range(len(df_pca_em)):
    plt.text(
        df_pca_em.PC1[i], 
        df_pca_em.PC2[i], 
        df_pca_em.emoji[i], 
        fontsize=14, # Grandezza emoji
        ha='center', va='center'
    )

plt.title('Spazio Semantico: Analisi specifica delle Emoji', fontsize=16)
plt.xlabel(f'PC1 ({pca_em.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca_em.explained_variance_ratio_[1]:.1%})')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', title="Emozione Dominante")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

NameError: name 'os' is not defined

## Visualizzazione delle Parole (PCA e TruncatedSVD)


### Standardizzazione e Applicazione della PCA
La PCA richiede che i dati siano centrati (media 0) e scalati (varianza unitaria).

### Visualizzazione PCA (Il Grafico)


### Applicazione di TruncatedSVD
La TruncatedSVD è un'alternativa alla PCA che può essere più efficiente per matrici sparse.

### Interpretazione
La visualizzazione PCA e SVD mostra come le parole si raggruppano in base alle emozioni predominanti.
Le parole con emozioni simili tendono a essere vicine nello spazio 2D, indicando che il lessico ELIta cattura efficacemente le relazioni semantiche basate sulle emozioni.
Analizzando i cluster, possiamo osservare:
* Cluster di "gioia" e "fiducia" spesso vicini, riflettendo emozioni positive correlate.
* Cluster di "rabbia", "disgusto" e "paura" che tendono a raggrupparsi, indicando emozioni negative correlate.
